# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 1
iteration = "-5"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 5
num_batch = 2000 #25000 #11112
val_batch = 20
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

#             if save_masks and phase == 'val':
#                 outputs = sigmoid_function(outputs)
#                 # need to fix before running
#                 outputs_dir = "data/outputs/results/" + str(datetime.now().date())
#                 os.makedirs(outputs_dir, exist_ok=True)
#                 threshold = 0.05
#                 j = 0
#                 for locations in paths:
#                     img = Image.open(locations)
#                     #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     for threshold in [0.05, .25, .5, .75]:
#                         img = data_factory.showMaskOnImage(img,outputs[j], threshold)
#                         img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
#                     j += 1

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
            #torch.save(model, SAVE_LOCATION)
            #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    #torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    #torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/1
Batch 1/2000
loss: 0.40258315205574036
---------------
Batch 2/2000
loss: 0.3849811553955078
---------------
Batch 3/2000
loss: 0.35461166501045227
---------------
Batch 4/2000
loss: 0.4434349536895752
---------------
Batch 5/2000
loss: 0.2824491262435913
---------------
Batch 6/2000
loss: 0.35162353515625
---------------
Batch 7/2000
loss: 0.35948944091796875
---------------
Batch 8/2000
loss: 0.4272395074367523
---------------
Batch 9/2000
loss: 0.34495246410369873
---------------
Batch 10/2000
loss: 0.3176911771297455
---------------
Batch 11/2000
loss: 0.3993465304374695
---------------
Batch 12/2000
loss: 0.37545689940452576
---------------
Batch 13/2000
loss: 0.3045191764831543
---------------
Batch 14/2000
loss: 0.40502235293388367
---------------
Batch 15/2000
loss: 0.38070037961006165
---------------
Batch 16/2000
loss: 0.33527636528015137
---------------
Batch 17/2000
loss: 0.3552585542201996
---------------
Batch 18/2000
loss: 0.36182358860969543
---------------
Ba

loss: 0.35313305258750916
---------------
Batch 149/2000
loss: 0.3653876781463623
---------------
Batch 150/2000
loss: 0.40405288338661194
---------------
Batch 151/2000
loss: 0.3807050585746765
---------------
Batch 152/2000
loss: 0.2964506149291992
---------------
Batch 153/2000
loss: 0.35706058144569397
---------------
Batch 154/2000
loss: 0.3367880880832672
---------------
Batch 155/2000
loss: 0.43500471115112305
---------------
Batch 156/2000
loss: 0.38911348581314087
---------------
Batch 157/2000
loss: 0.33067214488983154
---------------
Batch 158/2000
loss: 0.3027952313423157
---------------
Batch 159/2000
loss: 0.32399100065231323
---------------
Batch 160/2000
loss: 0.373132586479187
---------------
Batch 161/2000
loss: 0.3268497586250305
---------------
Batch 162/2000
loss: 0.4236173629760742
---------------
Batch 163/2000
loss: 0.3252510130405426
---------------
Batch 164/2000
loss: 0.42903584241867065
---------------
Batch 165/2000
loss: 0.2926860749721527
---------------


---------------
Batch 294/2000
loss: 0.30086106061935425
---------------
Batch 295/2000
loss: 0.46884870529174805
---------------
Batch 296/2000
loss: 0.3726661503314972
---------------
Batch 297/2000
loss: 0.4868289828300476
---------------
Batch 298/2000
loss: 0.265438050031662
---------------
Batch 299/2000
loss: 0.38805797696113586
---------------
Batch 300/2000
loss: 0.2728216350078583
---------------
Batch 301/2000
loss: 0.47443312406539917
---------------
Batch 302/2000
loss: 0.44511064887046814
---------------
Batch 303/2000
loss: 0.3873920440673828
---------------
Batch 304/2000
loss: 0.36596953868865967
---------------
Batch 305/2000
loss: 0.3650195598602295
---------------
Batch 306/2000
loss: 0.33769354224205017
---------------
Batch 307/2000
loss: 0.3622322976589203
---------------
Batch 308/2000
loss: 0.3332119286060333
---------------
Batch 309/2000
loss: 0.40205979347229004
---------------
Batch 310/2000
loss: 0.38247746229171753
---------------
Batch 311/2000
loss: 0.4

loss: 0.34465137124061584
---------------
Batch 440/2000
loss: 0.41468435525894165
---------------
Batch 441/2000
loss: 0.31890323758125305
---------------
Batch 442/2000
loss: 0.3704909682273865
---------------
Batch 443/2000
loss: 0.3442418575286865
---------------
Batch 444/2000
loss: 0.2319667637348175
---------------
Batch 445/2000
loss: 0.27753952145576477
---------------
Batch 446/2000
loss: 0.29769018292427063
---------------
Batch 447/2000
loss: 0.4473917484283447
---------------
Batch 448/2000
loss: 0.389534056186676
---------------
Batch 449/2000
loss: 0.2899722456932068
---------------
Batch 450/2000
loss: 0.4058000445365906
---------------
Batch 451/2000
loss: 0.34532809257507324
---------------
Batch 452/2000
loss: 0.39364051818847656
---------------
Batch 453/2000
loss: 0.417011022567749
---------------
Batch 454/2000
loss: 0.3321606516838074
---------------
Batch 455/2000
loss: 0.3540845811367035
---------------
Batch 456/2000
loss: 0.4110313653945923
---------------
Ba

---------------
Batch 585/2000
loss: 0.35105159878730774
---------------
Batch 586/2000
loss: 0.3972387909889221
---------------
Batch 587/2000
loss: 0.2754887044429779
---------------
Batch 588/2000
loss: 0.42722102999687195
---------------
Batch 589/2000
loss: 0.39409375190734863
---------------
Batch 590/2000
loss: 0.3446202278137207
---------------
Batch 591/2000
loss: 0.28064998984336853
---------------
Batch 592/2000
loss: 0.4055052101612091
---------------
Batch 593/2000
loss: 0.42061081528663635
---------------
Batch 594/2000
loss: 0.45773929357528687
---------------
Batch 595/2000
loss: 0.30016380548477173
---------------
Batch 596/2000
loss: 0.3515181839466095
---------------
Batch 597/2000
loss: 0.3811012804508209
---------------
Batch 598/2000
loss: 0.45748716592788696
---------------
Batch 599/2000
loss: 0.31485721468925476
---------------
Batch 600/2000
loss: 0.35563668608665466
---------------
Batch 601/2000
loss: 0.3933963477611542
---------------
Batch 602/2000
loss: 0

...

---------------
Batch 1307/2000
loss: 0.41485029458999634
---------------
Batch 1308/2000
loss: 0.3327602446079254
---------------
Batch 1309/2000
loss: 0.38273561000823975
---------------
Batch 1310/2000
loss: 0.42897269129753113
---------------
Batch 1311/2000
loss: 0.3340398371219635
---------------
Batch 1312/2000
loss: 0.5257454514503479
---------------
Batch 1313/2000
loss: 0.3143085837364197
---------------
Batch 1314/2000
loss: 0.4834698736667633
---------------
Batch 1315/2000
loss: 0.4862210154533386
---------------
Batch 1316/2000
loss: 0.39893800020217896
---------------
Batch 1317/2000
loss: 0.4091794788837433
---------------
Batch 1318/2000
loss: 0.4592303931713104
---------------
Batch 1319/2000
loss: 0.3523724377155304
---------------
Batch 1320/2000
loss: 0.4145832061767578
---------------
Batch 1321/2000
loss: 0.38646623492240906
---------------
Batch 1322/2000
loss: 0.45990899205207825
---------------
Batch 1323/2000
loss: 0.41705676913261414
---------------
Batch 13

---------------
Batch 1450/2000
loss: 0.38623666763305664
---------------
Batch 1451/2000
loss: 0.485524445772171
---------------
Batch 1452/2000
loss: 0.3751046657562256
---------------
Batch 1453/2000
loss: 0.4247303903102875
---------------
Batch 1454/2000
loss: 0.24862052500247955
---------------
Batch 1455/2000
loss: 0.4209270775318146
---------------
Batch 1456/2000
loss: 0.32936087250709534
---------------
Batch 1457/2000
loss: 0.3917634189128876
---------------
Batch 1458/2000
loss: 0.3517819344997406
---------------
Batch 1459/2000
loss: 0.3688108026981354
---------------
Batch 1460/2000
loss: 0.40854376554489136
---------------
Batch 1461/2000
loss: 0.4099770188331604
---------------
Batch 1462/2000
loss: 0.32470548152923584
---------------
Batch 1463/2000
loss: 0.44043871760368347
---------------
Batch 1464/2000
loss: 0.2884988486766815
---------------
Batch 1465/2000
loss: 0.28181156516075134
---------------
Batch 1466/2000
loss: 0.3649318814277649
---------------
Batch 146

---------------
Batch 1593/2000
loss: 0.3714536726474762
---------------
Batch 1594/2000
loss: 0.3966800570487976
---------------
Batch 1595/2000
loss: 0.35394302010536194
---------------
Batch 1596/2000
loss: 0.3322294056415558
---------------
Batch 1597/2000
loss: 0.33977100253105164
---------------
Batch 1598/2000
loss: 0.3900122046470642
---------------
Batch 1599/2000
loss: 0.36378780007362366
---------------
Batch 1600/2000
loss: 0.38666290044784546
---------------
Batch 1601/2000
loss: 0.36076897382736206
---------------
Batch 1602/2000
loss: 0.3173857629299164
---------------
Batch 1603/2000
loss: 0.3764987885951996
---------------
Batch 1604/2000
loss: 0.3536326289176941
---------------
Batch 1605/2000
loss: 0.4089074432849884
---------------
Batch 1606/2000
loss: 0.40490686893463135
---------------
Batch 1607/2000
loss: 0.29305723309516907
---------------
Batch 1608/2000
loss: 0.323638916015625
---------------
Batch 1609/2000
loss: 0.3670160174369812
---------------
Batch 161

---------------
Batch 1736/2000
loss: 0.3536415696144104
---------------
Batch 1737/2000
loss: 0.43902233242988586
---------------
Batch 1738/2000
loss: 0.3806631863117218
---------------
Batch 1739/2000
loss: 0.38496607542037964
---------------
Batch 1740/2000
loss: 0.21844619512557983
---------------
Batch 1741/2000
loss: 0.3860976994037628
---------------
Batch 1742/2000
loss: 0.3316548466682434
---------------
Batch 1743/2000
loss: 0.3623085021972656
---------------
Batch 1744/2000
loss: 0.3521791994571686
---------------
Batch 1745/2000
loss: 0.36656543612480164
---------------
Batch 1746/2000
loss: 0.43731608986854553
---------------
Batch 1747/2000
loss: 0.4587073028087616
---------------
Batch 1748/2000
loss: 0.30425065755844116
---------------
Batch 1749/2000
loss: 0.3438340723514557
---------------
Batch 1750/2000
loss: 0.3361484408378601
---------------
Batch 1751/2000
loss: 0.3456334173679352
---------------
Batch 1752/2000
loss: 0.3074600398540497
---------------
Batch 175

Batch 1879/2000
loss: 0.3938697874546051
---------------
Batch 1880/2000
loss: 0.33099091053009033
---------------
Batch 1881/2000
loss: 0.2829478085041046
---------------
Batch 1882/2000
loss: 0.39455804228782654
---------------
Batch 1883/2000
loss: 0.40230756998062134
---------------
Batch 1884/2000
loss: 0.5154485106468201
---------------
Batch 1885/2000
loss: 0.3811022937297821
---------------
Batch 1886/2000
loss: 0.355778306722641
---------------
Batch 1887/2000
loss: 0.41513097286224365
---------------
Batch 1888/2000
loss: 0.5087472200393677
---------------
Batch 1889/2000
loss: 0.5023159384727478
---------------
Batch 1890/2000
loss: 0.3965783417224884
---------------
Batch 1891/2000
loss: 0.4759639501571655
---------------
Batch 1892/2000
loss: 0.3850651681423187
---------------
Batch 1893/2000
loss: 0.3766152858734131
---------------
Batch 1894/2000
loss: 0.37284260988235474
---------------
Batch 1895/2000
loss: 0.4265159070491791
---------------
Batch 1896/2000
loss: 0.466

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [8]:
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(